In [ ]:
import pandas as pd
import numpy as np
import importlib
import os
from Utils import model_data_engineering
from Utils import preprocessing 
from Utils import plotting

# reload
importlib.reload(model_data_engineering)
importlib.reload(preprocessing)
importlib.reload(plotting)


# importing modules
from Utils.preprocessing import combine_csv_files
from Utils.preprocessing import perform_downsampling
from Utils.plotting import plot
from Utils.featureEngineering import creating_more_features
from sklearn.model_selection import train_test_split


from Utils import Evaluation
importlib.reload(Evaluation)


from Utils import model_data_engineering
importlib.reload(model_data_engineering)

import numpy as np
import os
from sklearn.model_selection import train_test_split


In [ ]:
# # # Example usage
# df = combine_csv_files('../raw_data/', '../preprocessed_data/', saveFile=True)
# df.head()

In [ ]:
# Example usage
df = pd.read_csv('../preprocessed_data/combined.csv')

In [ ]:
hourly_df = perform_downsampling(df, freq='1H',aggregation_func='mean')


In [ ]:
x_feature_name = 'Timestamp'
y_feature_name = 'Summe'
title = 'Hourly Plot '
plot(hourly_df,x_feature_name,y_feature_name,title = title,preview=True)

# Custom Model

In [ ]:
features_df = creating_more_features(hourly_df,'Timestamp')
features_df['TimeOfDay'] = features_df['TimeOfDay'].map({'Night': 0, 'Morning': 1, 'Afternoon': 2, 'Evening': 3})


features_df.head()

In [ ]:

holiday_df = features_df[features_df['Holiday']==1]



weekned_df = features_df[features_df['Weekend']==1]


Night_df = features_df[features_df['TimeOfDay']==0]
Morning_df = features_df[features_df['TimeOfDay']==1]
evening_Df = features_df[features_df['TimeOfDay']==3]
afternoon_df = features_df[features_df['TimeOfDay']==2]
monday_df = features_df[features_df['DayOfWeek']==0]
tuesday_df = features_df[features_df['DayOfWeek']==1]
wedness_df = features_df[features_df['DayOfWeek']==2]
thurday_df = features_df[features_df['DayOfWeek']==3]
friday_df = features_df[features_df['DayOfWeek']==4]


# Visualization of different types of Segmentation

In [ ]:
import plotly.graph_objects as go  # Library for creating interactive plots
import numpy as np
import os
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=features_df['Timestamp'],
        y=features_df['Summe'],
        name=y_feature_name,
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=holiday_df['Timestamp'],
        y=holiday_df['Summe'],
        name='Public Holiday',
        mode='markers'
    ))
fig.add_trace(
    go.Scatter(
        x=weekned_df['Timestamp'],
        y=weekned_df['Summe'],
        name='Saturday+Sunday',
        mode='markers'
    ))

fig.add_trace(
    go.Scatter(
        x=Night_df['Timestamp'],
        y=Night_df['Summe'],
        name='Night Time',
        mode='markers'
    ))

fig.add_trace(
    go.Scatter(
        x=Morning_df['Timestamp'],
        y=Morning_df['Summe'],
        name='Morning Time',
        mode='markers'
    ))

fig.add_trace(
    go.Scatter(
        x=evening_Df['Timestamp'],
        y=evening_Df['Summe'],
        name='EveningTime',
        mode='markers'
    ))

fig.add_trace(
    go.Scatter(
        x=afternoon_df['Timestamp'],
        y=afternoon_df['Summe'],
        name='AfterNoon',
        mode='markers'
    ))
fig.add_trace(
    go.Scatter(
        x=monday_df['Timestamp'],
        y=monday_df['Summe'],
        name='Monday',
        mode='markers'
    ))


fig.add_trace(
    go.Scatter(
        x=tuesday_df['Timestamp'],
        y=tuesday_df['Summe'],
        name='Tuesday',
        mode='markers'
    ))

fig.add_trace(
    go.Scatter(
        x=wedness_df['Timestamp'],
        y=wedness_df['Summe'],
        name='Wednessday',
        mode='markers'
    ))

fig.add_trace(
    go.Scatter(
        x=thurday_df['Timestamp'],
        y=thurday_df['Summe'],
        name='Thursday',
        mode='markers'
    ))
fig.add_trace(
    go.Scatter(
        x=friday_df['Timestamp'],
        y=friday_df['Summe'],
        name='Friday',
        mode='markers'
    ))



# Update xaxis properties
# The x-axis and y-axis titles are updated using the update_xaxes() and update_yaxes() methods of the figure object.
fig.update_xaxes(title_text='Date')

# Update yaxis properties
fig.update_yaxes(title_text=y_feature_name)

# Update title and height
# The layout of the figure is updated using the update_layout() method. The title, height, and width of the plot are set.
fig.update_layout(
    title=f'{title}',
    height=500,
    width=1200
)

fig.write_html('DetailVisualization_of_hourlyData.html')


In [ ]:
train , test =  train_test_split(features_df,train_size=0.8,shuffle=False)

In [ ]:
df = train 
df[df['Holiday']==1]

In [ ]:
import pandas as pd

def predict_next_day(df, dateOfPrediction):
    
    requiredData = df.loc[(df['Timestamp'] < dateOfPrediction)]
    if requiredData.shape[0] >= 24*7:
        

        is_weekend = df.loc[df['Timestamp'] == dateOfPrediction, ['Weekend','DayOfWeek']].iloc[0]
        is_holiday = df.loc[df['Timestamp'] == dateOfPrediction, ['Timestamp','Holiday']].iloc[0]
        # display(is_holiday)
        if is_weekend['DayOfWeek']==6:
            print('You want to predict Sunday')
            previousSunday = df.loc[(df['Timestamp'] < dateOfPrediction) & (df['DayOfWeek'] == 6)].tail(24)
            previousSunday['Timestamp'] = df['Timestamp'] +  pd.DateOffset(days=7)

            return previousSunday['Timestamp'] , previousSunday['Summe']
            display(previousSunday)

        elif is_weekend['DayOfWeek']==5:
            print('You want to predcict Saturday')
            previousSaturday = df.loc[(df['Timestamp'] < dateOfPrediction) & (df['DayOfWeek'] == 5)].tail(24)
            previousSaturday['Timestamp'] = df['Timestamp'] +  pd.DateOffset(days=7)

            # display(previousSaturday)
            return previousSaturday['Timestamp'],previousSaturday['Summe']
        
        elif is_holiday['Holiday']==1:
            print('Its a holiday')
            # previousHoliday = df.loc[ ((df['Timestamp'] < dateOfPrediction) & (df['Holiday'] == 1) ) 
            #                             or (df['Timestamp'] < dateOfPrediction) & (df['DayOfWeek'] == 5)  
            #                             or (df['Timestamp'] < dateOfPrediction) & (df['DayOfWeek'] == 6) 
            #                         ].tail(24)
            # previousHoliday['Timestamp'] = df['Timestamp'] +  pd.DateOffset(days=7)
            previous_holiday = df.loc[
                ((df['Timestamp'] < dateOfPrediction) & (df['Holiday'] == 1)) |
                ((df['Timestamp'] < dateOfPrediction) & ((df['DayOfWeek'] == 5) | (df['DayOfWeek'] == 6)))
            ].tail(24)

            previous_holiday = previous_holiday.copy()  # Create a copy of the filtered DataFrame
            previous_holiday['Timestamp'] = previous_holiday['Timestamp'] + pd.DateOffset(days=7)


            # display(previousHoliday)
            return previous_holiday['Timestamp'] ,previous_holiday['Summe']


        else:
            print('Required Day is normal day')
            previousDay = df.loc[df['Timestamp'] == (dateOfPrediction - pd.DateOffset(days=1)), ['Summe', 'Timestamp']]
            if previousDay.empty:
                print('Your Previous Day is empty 😔')
            else:
                previousDayDate = previousDay['Timestamp'].values[0]
                print('PreviousDayDate is', previousDayDate)

                # Correct the indexing and filtering of previousWholeDay.
                previousWholeDay = df[(df['Timestamp'] >= previousDayDate) & (df['Timestamp'] < dateOfPrediction)]
                previousWholeDay['Timestamp'] = df['Timestamp'] +  pd.DateOffset(days=1)
                # display(previousWholeDay)
                # previousWholeDay.plot(x='Timestamp',y='Summe')
                return previousWholeDay['Timestamp'],previousWholeDay['Summe']
                
    else:
        print('You want not enough data in the past. Please move required date ahead')

# Assuming df is your DataFrame with 'Timestamp' and 'Summe' columns.
# df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Example usage
# predict_next_day(df, pd.to_datetime('2014-03-21 00:00:00'))


In [93]:
resultDic = {
    'PredictedDate':[],
    'PredictedEEConsumption':[],
    'ActualfilteredDf':[]
}
def filter_by_timestamp(df, date_list):
    filtered_df = df[df['Timestamp'].isin(date_list)]
    return filtered_df

# actual_timestamps = ['2014-03-12 07:00:00', '2014-03-12 08:00:00', '2015-03-26 18:00:00']

# result_df = filter_by_timestamp(df, actual_timestamps)
print(result_df)
for day in range(0,24*90):
    interestedDate = pd.to_datetime('2014-04-21 00:00:00') +pd.DateOffset(day=day)
    print('InterestedDate:', interestedDate)
    date , prediction = predict_next_day(df, interestedDate)
    filteredDf =  filter_by_timestamp(df, date.tolist())
    resultDic['PredictedDate'].append(date.tolist())
    resultDic['PredictedEEConsumption'].append(prediction.tolist())
    
  
    resultDic['ActualfilteredDf'].append(  filteredDf['Summe'].tolist())

    
    # break


               Timestamp       Summe  Year  Month  Day  Hour  DayOfWeek  \
0    2014-03-12 07:00:00   66.321410  2014      3   12     7          2   
1    2014-03-12 08:00:00   14.977094  2014      3   12     8          2   
9107 2015-03-26 18:00:00  848.802540  2015      3   26    18          3   

      Weekend  Holiday TimeOfDay  
0           0        0         1  
1           0        0         1  
9107        0        0         2  
InterestedDate: 2014-04-21 00:00:00
Its a holiday
InterestedDate: 2014-04-01 00:00:00
Required Day is normal day
PreviousDayDate is 2014-03-31T00:00:00.000000000
InterestedDate: 2014-04-02 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-01T00:00:00.000000000
InterestedDate: 2014-04-03 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-02T00:00:00.000000000
InterestedDate: 2014-04-04 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-03T00:00:00.000000000
InterestedDate: 2014-04-05 00:00:00
You want to predcict Satu

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykerne

Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 0

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykerne

Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 0

...

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykerne

InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.00

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykerne

InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.000000000
InterestedDate: 2014-04-30 00:00:00
Required Day is normal day
PreviousDayDate is 2014-04-29T00:00:00.00

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_14589/1369237070.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykerne

In [81]:
import numpy as np

def Evaluation(actual_values, predicted_values):
    """
    Calculate Mean Absolute Error (MAE).

    Parameters:
        actual_values (array-like): Array or pandas Series containing the actual values.
        predicted_values (array-like): Array or pandas Series containing the predicted values.

    Returns:
        float: The Mean Absolute Error (MAE).
    """
    mae = np.mean(np.abs(actual_values - predicted_values))
    mape = np.mean(np.abs((actual_values - predicted_values) / actual_values)) * 100


    return mae , mape



In [94]:
MAE_LIST  = []
MAPE_LIST = []
for i in range(0,24*90):
    meanAbsoluteError ,  MeanAbsolutePercentageError = Evaluation(np.array(resultDic['PredictedEEConsumption'][i]) , np.array(resultDic['ActualfilteredDf'][i]))
    MAE_LIST.append(meanAbsoluteError)
    MAPE_LIST.append(MeanAbsolutePercentageError)
    
    # print(f"'MAE : {meanAbsoluteError} MAPE:{MeanAbsolutePercentageError}")
    
print('Average MAE :',np.mean(MAE_LIST))
print('Average MAPE :',np.mean(MAPE_LIST))


Average MAE : 58.694291852179944
Average MAPE : 84.47014331979818


In [84]:
resultDic['PredictedEEConsumption'][0]

[31.10758508522065,
 18.47105514060929,
 24.614799226269266,
 24.118723990741845,
 18.388868337328944,
 31.748451669826192,
 18.068476462135074,
 16.909515651874678,
 30.70638807721905,
 16.94222676627396,
 28.07008291356347,
 20.250275955703746,
 17.01867634442364,
 29.96105271894451,
 17.480338910520846,
 28.177920839114655,
 21.112525275620918,
 17.24128994013267,
 30.685853257632804,
 16.74559417547406,
 29.383970937676793,
 19.59984400043885,
 17.77314103681162,
 30.291984651577724]

In [83]:
resultDic['ActualfilteredDf'][0]

[17.423719782796113,
 26.921552326483816,
 21.04175123713027,
 20.33664854935726,
 27.460860138082353,
 18.713342761179987,
 31.13541842516195,
 88.9135117194441,
 77.61108244582394,
 27.51785559085829,
 17.248687205759307,
 23.077571876024347,
 14.956745166882442,
 26.669165143793293,
 35.94508144705816,
 249.85308158433108,
 81.52003732708904,
 31.050079747825407,
 25.70040707019658,
 21.24878189426106,
 43.525791035478974,
 18.480901234213743,
 30.616470151833262,
 17.577223185227773]

In [ ]:
import pandas as pd
import plotly.graph_objects as go

def filter_by_timestamp(df, date_list):
    filtered_df = df[df['Timestamp'].isin(date_list)]
    return filtered_df

def plot_result_dic(resultDic):
    fig = go.Figure()

    for idx, predicted_date in enumerate(resultDic['PredictedDate']):
        trace_predicted = go.Scatter(x=predicted_date, y=resultDic['PredictedEEConsumption'][idx], mode='lines', name='Predicted')
        trace_actual = go.Scatter(x=predicted_date, y=resultDic['ActualfilteredDf'][idx], mode='lines+markers', name='Actual')

        fig.add_trace(trace_predicted)
        fig.add_trace(trace_actual)

    fig.update_layout(title='Predicted vs. Actual Energy Consumption',
                      xaxis_title='Timestamp',
                      yaxis_title='Energy Consumption')

    fig.show()

resultDic = {
    'PredictedDate': [],
    'PredictedEEConsumption': [],
    'ActualfilteredDf': []
}

for day in range(0, 30):
    interestedDate = pd.to_datetime('2014-04-21 00:00:00') + pd.DateOffset(days=day)
    print('InterestedDate:', interestedDate)
    date, prediction = predict_next_day(df, interestedDate)
    filteredDf = filter_by_timestamp(df, date.tolist())
    resultDic['PredictedDate'].append(date.tolist())
    resultDic['PredictedEEConsumption'].append(prediction.tolist())
    resultDic['ActualfilteredDf'].append(filteredDf['Summe'].tolist())

# Call the function to plot the data in resultDic
plot_result_dic(resultDic)


In [ ]:
df

In [ ]:
# resultDic['PredictedDate'] ,resultDic['EEConsumption']
import matplotlib.pyplot as plt
predicted_date = resultDic['PredictedDate']
predicted_consumption = resultDic['EEConsumption']
for i in range(0,len(resultDic['PredictedDate'])):
    plt.plot(x=predicted_date[i],y=predicted_consumption[i])

In [ ]:
for key ,value in resultDic.items():
    print(key)

In [ ]:
resultDic['PredictedDate']

In [ ]:
import pandas as pd
import numpy as np
import importlib
import os
from Utils import model_data_engineering
from Utils import preprocessing 
from Utils import plotting

# reload
importlib.reload(model_data_engineering)
importlib.reload(preprocessing)
importlib.reload(plotting)


# importing modules
from Utils.preprocessing import combine_csv_files
from Utils.preprocessing import perform_downsampling
from Utils.plotting import plot
from Utils.featureEngineering import creating_more_features
from sklearn.model_selection import train_test_split


from Utils import Evaluation
importlib.reload(Evaluation)


from Utils import model_data_engineering
importlib.reload(model_data_engineering)

import numpy as np
import os
from sklearn.model_selection import train_test_split


# Testing for Holidays

In [ ]:
df

In [ ]:
df[df['DayOfWeek']==2]

In [ ]:
import pandas as pd

# Assuming you have already read the DataFrame into a variable named 'df'
# If not, you can read it using: df = pd.read_csv('your_data.csv')

# Step 1: Preprocess the data (if needed)
# For example, convert 'Timestamp' column to datetime dtype
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Step 2: Create a function to predict the next day's value
def predict_next_day(df, current_date):
    # Check if the current_date is a holiday or weekend (Saturday or Sunday)
    is_holiday = df.loc[df['Timestamp'] == current_date, 'Holiday'].iloc[0]
    is_weekend = df.loc[df['Timestamp'] == current_date, 'Weekend'].iloc[0]
    
    # If it's a holiday or weekend, find the previous holiday of the same type
    if is_holiday or is_weekend:
        previous_holiday = df.loc[(df['Timestamp'] < current_date) & (df['Holiday'] == 1)].tail(1)
        if not previous_holiday.empty:
            return previous_holiday['Summe'].iloc[0]
        else:
            # If there's no previous holiday of the same type, fall back to the last Sunday (if predicting Saturday) or Saturday (if predicting Sunday)
            if current_date.weekday() == 6:  # Sunday
                previous_holiday = df.loc[(df['Timestamp'] < current_date) & (df['DayOfWeek'] == 6)].tail(1)
            else:
                previous_holiday = df.loc[(df['Timestamp'] < current_date) & (df['DayOfWeek'] == 5)].tail(1)
            if not previous_holiday.empty:
                return previous_holiday['Summe'].iloc[0]
    
    # If it's a regular day, check if there's a previous day in the DataFrame
    previous_day = df.loc[df['Timestamp'] == (current_date - pd.DateOffset(days=1)), 'Summe']
    if not previous_day.empty:
        display(previous_day)
        return previous_day.iloc[0]
    else:
        return None  # Handle the case when there's no previous day data

# Step 4: Iterate through the DataFrame to make predictions for each day
predictions = []
for index, row in df.iterrows():
    current_date = row['Timestamp']
    print('Current Date : ',current_date)
    prediction = predict_next_day(df, current_date)
    print('Preduction : ',prediction)
    predictions.append(prediction)

# Add the predictions to the DataFrame
df['NextDayPrediction'] = predictions

# Print or store the DataFrame with the predictions
# print(df)


In [ ]:
predictions

In [ ]:
import numpy as np

def give_me_historical_values(train, test):
    test['Predicted_mean'] = np.float64('nan')
    test['Predicted_median'] = np.float64('nan')
    test['predicted_monthly_mean'] = np.float64('nan') 
    test['predicted_monthly_median'] = np.float64('nan') 
    test['predicted_base_on_lag1'] =  np.float64('nan')
    
    for index, row in test.iterrows():
        hour = row['Hour']
        dayofWeek = row['DayOfWeek']
        weekend = row['Weekend']
        timeOfDay = row['TimeOfDay']
        day = row['Day']
        month = row['Month']
        Year = row['Year']

        # print(f'Hour: {hour}\nDayOfWeek: {dayofWeek}\nWeekend: {weekend}\nTimeOfDay: {timeOfDay}\nDay: {day}\nMonth:{month}\nYear:{Year}')
        train_df = train[
            (train['Hour'] == hour) &
            (train['DayOfWeek'] == dayofWeek) &
            (train['Weekend'] == weekend) &
            (train['TimeOfDay'] == timeOfDay) &
            (train['Day'] == day) 
        ]
        
        last_consective_day = train[
            (train['Hour'] == hour) &
            (train['DayOfWeek'] == dayofWeek) &
            (train['Weekend'] == weekend) &
            (train['TimeOfDay'] == timeOfDay)
        ]
    
        
        display(last_consective_day)
        train_df_monthly = train[
            (train['Hour'] == hour) &
            (train['DayOfWeek'] == dayofWeek) &
            (train['Weekend'] == weekend) &
            (train['TimeOfDay'] == timeOfDay) &
            (train['Month'] == month) 
        ]
        
        # display(last_consective_day)
        
        hourly_values_mean = train_df['Summe'].mean()
        hourly_values_median = train_df['Summe'].median()
        
        monthly_values_mean = train_df_monthly['Summe'].mean()
        monthly_values_median = train_df_monthly['Summe'].median()

        test.at[index, 'Predicted_mean'] = hourly_values_mean
        test.at[index, 'Predicted_median'] = hourly_values_median

        test.at[index, 'predicted_monthly_mean'] = monthly_values_mean
        test.at[index, 'predicted_monthly_median'] = monthly_values_median
        
        test.at[index, 'predicted_base_on_lag1'] = last_consective_day.iloc[-1].Summe
        
    return test 


def give_me_yesterday_historical(train, test):
    test['Predicted'] = np.float64('nan')
    
    for index, row in test.iterrows():
        hour = row['Hour']
        dayofWeek = row['DayOfWeek']
        weekend = row['Weekend']
        timeOfDay = row['TimeOfDay']
        day = row['Day']
        month = row['Month']
        Year = row['Year']



        # checking conditions for public holiday
        try:
            if dayofWeek in [5,6]:
                # print('Holiday')
                # print(f'Hour: {hour}\nDayOfWeek: {dayofWeek}\nWeekend: {weekend}\nTimeOfDay: {timeOfDay}\nDay: {day}\nMonth:{month}\nYear:{Year}')

                # print('Day of Week :',dayofWeek)
                train_df = train[
                    (train['Hour'] == hour) &
                    (train['DayOfWeek'] == dayofWeek) 
                ]
                # display(train_df)
                test.at[index, 'Predicted'] = train_df.iloc[-1].Summe
            else:
                # print('Not Holiday')
                # print(f'Hour: {hour}\nDayOfWeek: {dayofWeek}\nWeekend: {weekend}\nTimeOfDay: {timeOfDay}\nDay: {day}\nMonth:{month}\nYear:{Year}')

                train_df = train[
                    (train['Hour'] == hour) &
                    (train['DayOfWeek'] == dayofWeek) &
                    (train['Day'] == day-1) # look for the yesterday

                ]
                # display(train_df)
                test.at[index, 'Predicted'] = train_df.iloc[-1].Summe
        except:
            pass
    return test 


In [ ]:
result_df= give_me_yesterday_historical(train,test)


In [ ]:
import plotly.graph_objects as go  # Library for creating interactive plots
import numpy as np
import os
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['Summe'],
        name='Actual',
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['Predicted'],
        name='Predicted',
        mode='lines+markers'
    ))







# Update xaxis properties
# The x-axis and y-axis titles are updated using the update_xaxes() and update_yaxes() methods of the figure object.
fig.update_xaxes(title_text='Date')

# Update yaxis properties
fig.update_yaxes(title_text='Summe')

# Update title and height
# The layout of the figure is updated using the update_layout() method. The title, height, and width of the plot are set.
fig.update_layout(
    title=f'{title}',
    height=500,
    width=1200
)
fig.show()
print('by yesterday comparison : ',Evaluation(result_df['Summe'],result_df['Predicted']))


# fig.write_html('Results.html')


In [ ]:
import plotly.graph_objects as go  # Library for creating interactive plots
import numpy as np
import os
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['Summe'],
        name='Actual',
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['Predicted_mean'],
        name='Predicted by Mean',
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['Predicted_median'],
        name='Predicted by Medain',
        mode='lines+markers'
    ))

fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['predicted_monthly_mean'],
        name='added Monthly-Predicted by Median',
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['predicted_monthly_median'],
        name='added Monthly-Predicted by Mean',
        mode='lines+markers'
    ))

fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['predicted_base_on_lag1'],
        name='lag-1',
        mode='lines+markers'
    ))







# Update xaxis properties
# The x-axis and y-axis titles are updated using the update_xaxes() and update_yaxes() methods of the figure object.
fig.update_xaxes(title_text='Date')

# Update yaxis properties
fig.update_yaxes(title_text='Summe')

# Update title and height
# The layout of the figure is updated using the update_layout() method. The title, height, and width of the plot are set.
fig.update_layout(
    title=f'{title}',
    height=500,
    width=1200
)


fig.write_html('Results.html')


# Evaluate

In [ ]:
import numpy as np

def Evaluation(actual_values, predicted_values):
    """
    Calculate Mean Absolute Error (MAE).

    Parameters:
        actual_values (array-like): Array or pandas Series containing the actual values.
        predicted_values (array-like): Array or pandas Series containing the predicted values.

    Returns:
        float: The Mean Absolute Error (MAE).
    """
    mae = np.mean(np.abs(actual_values - predicted_values))
    mape = np.mean(np.abs((actual_values - predicted_values) / actual_values)) * 100


    return mae , mape

print('by Mean',Evaluation(value['Summe'],value['Predicted_mean']))
print('by Medain',Evaluation(value['Summe'],value['Predicted_median']))
print('by Mean(added Monthly)',Evaluation(value['Summe'],value['predicted_monthly_mean']))
print('by Mean(added Monthly)',Evaluation(value['Summe'],value['predicted_monthly_median']))
print('by lag-1',Evaluation(value['Summe'],value['predicted_base_on_lag1']))

